> **⚠️ CAUTION:**  
> <p style="color: red;">This project is work in progress. It can not be used for data analysis yet. Breathing and blood pressure data are totally untested/not yet implemented.</p>

This project is mainly the introduction of an library. The idea is that library can be used without knowledge of `python` by using the App.

# HRApp: An Interactive Heart Rate Variability (HRV) Analysis Tool

## Overview

The **HRApp** is an interactive tool designed to assist in the analysis of heart rate variability (HRV) data. It provides a graphical user interface (GUI) within Jupyter Notebooks to explore, preprocess, and visualize HRV data. The app organizes its features into multiple tabs, each focusing on a specific aspect of HRV analysis.

## Features

1. **PreProcessing Tab**  
   - Provides tools to preprocess HRV data, such as cleaning and inspecting inter-beat intervals (IBI).
   - Displays a customizable GUI for manipulating the dataset.

2. **Poincare Tab**  
   - Visualizes HRV data using Poincaré plots, which highlight the relationships between successive IBIs.
   - Ideal for assessing nonlinear dynamics in heart rate data.

3. **Descriptives Tab**  
   - Computes detailed descriptive statistics for IBIs, grouped by epochs.
   - Includes metrics such as mean, standard deviation, RMSSD, SDNN, SD1, SD2, and others.
   - Integrates power spectral density (PSD) results into the statistics, if available.

4. **PSD Tab**  
   - Uses Welch's method to compute power spectral density (PSD) for HRV data.
   - Groups PSD results by epochs, providing insights into the frequency domain features of HRV.

5. **Epochs Tab**  
   - Visualizes epochs of HRV data using a Gantt chart.
   - Offers a clear representation of time-based data segmentation.


In [1]:
import spectHR as cs
# include matplotlib **only** to set the style
import matplotlib.pyplot as plt

%matplotlib widget
plt.style.use('seaborn-v0_8-colorblind')

# Create the Log Output here:
cs.handler.show_logs()


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\P154492\AppData\Local\anaconda3\envs\Start\Lib\runpy.py", line 198, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\P154492\AppData\Local\anaconda3\envs\Start\Lib\runpy.py", line 88, in _run_code
    exec(code, run_globals)
  File "C:\Users\P154492\AppData\Local\anaconda3\envs\Start\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\P154492\AppData\Local\anaconda3\envs\Start\Lib\site-packages\traitlets\config\

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

# Demo

## Dataset loading:

Data that can be loaded are for now restricted to the [`xdf`](https://github.com/sccn/xdf) type used by [`labstreaminglayer`](https://labstreaminglayer.readthedocs.io/info/intro.html). The data tested are all recorded using the [`labrecorder`](https://github.com/labstreaminglayer/App-LabRecorder/releases), and streamed with the [`polarGUI`](https://github.com/markspan/PolarBand2lsl/releases/tag/v1.0.0) utility that streams data from the Polar H10 Band. Markers should be a seperate stream, with start- and stop markers. In the demo they are streamed through python inline code in [`opensesame`](https://osdoc.cogsci.nl/4.0/download). 

The call to `borderData` cuts the data from 1 second before the first marker, to one second after the last marker. To do any meaningfull analysis, the call to `calcPeaks` is mandatory: it attaches the DataFrame 'RTops' to hte DataSet. 

The code below creates a fileselector that shows all the .xdf nd all the .txt files in the current directory (minus TODO.txt). Selecting one will show the dataset as is now is: changes (edits) should be reflected immideately. If you want to continue editing later, this should be automatic. If you want to re-read the data you can opt to delete the pickle file from the cache directort, or change the `reset = False` to true in the preProc function below.



In [2]:

%matplotlib widget
import ipywidgets as widgets
import glob
import spectHR as cs  
from IPython.display import display

# Retrieve lists of available files
xdffiles = glob.glob("*.xdf")  # List all .xdf files
rawfiles = glob.glob("*.txt")  # List all .txt files
harness = glob.glob("*._csv")  # List all .txt files
# Exclude specific files that should not be processed
if "TODO.txt" in rawfiles:
    rawfiles.remove("TODO.txt")

# Create a file selection widget
fileselector = widgets.Select(
    options = xdffiles + rawfiles + harness,  # Available file options
    value = xdffiles[0] if xdffiles else None,  # Default selection (first .xdf file if available)
    description = 'File:',
    disabled = False,
    rows = 10,
)

# Function to preprocess ECG data
def preprocess_data(file_name):
    """
    Load and preprocess ECG data from the selected file.
    Steps:
    1. Load dataset with SpectHRDataset.
    2. Slice ECG time series (from 4s to 1000000s).
    3. Apply border correction.
    4. Apply high-pass filter (cutoff = 1 Hz).
    """
    dataset = cs.SpectHRDataset(file_name, reset=False, flip='auto')
    dataset.ecg = dataset.ecg.slicetime(80, 8000)
    dataset = cs.borderData(dataset)
    dataset = cs.filterECGData(dataset, {"filterType": "highpass", "cutoff": 1})
    return dataset

# Output widget to display the application interface
myApp = widgets.Output()

# Dictionary to store preprocessed datasets
data_store = {name: preprocess_data(name) for name in xdffiles + rawfiles + harness}

# Default dataset selection
DataSet = data_store[fileselector.value] if fileselector.value else None

# Event handler for file selection change
def on_file_change(change):
    """
    Handle changes in the file selection widget.
    Steps:
    1. Save the current dataset state.
    2. Update DataSet to the new selection.
    3. If peak data (RTops) is missing, compute peaks.
    4. Refresh and display the HRV analysis app.
    """
    global DataSet
    if change['type'] == 'change' and change['name'] == 'value':
        old_file = change['old']
        new_file = change['new']
        
        # Save the state of the previous dataset
        data_store[old_file] = DataSet
        
        # Load new dataset
        DataSet = data_store[new_file]
        
        # Compute peaks if not already calculated
        if not hasattr(DataSet, 'RTops'):
            DataSet = cs.calcPeaks(DataSet)
        
        # Display the HRV app
        with myApp:
            myApp.clear_output()
            display(cs.HRApp(DataSet))

# Attach event listener to file selection widget
fileselector.observe(on_file_change, names='value')


## Running the App

Call the app, with the fileselector as input. 

In [3]:
display(fileselector)
display(myApp)

Select(description='File:', options=('SUB_002.xdf', 'SUB_013.xdf', 'SUB_022.xdf', 'POLARDATA.txt', 'Data._csv'…

Output()

In [4]:
data = cs.SpectHRDataset('Data._csv', reset = True)
data2 = cs.SpectHRDataset('SUB_002.xdf', reset = True)

Magic is 0.6791066527366638


In [5]:
dataset = cs.filterECGData(data2, {"filterType": "highpass", "cutoff": 10})

In [6]:
data.ecg.srate

250

In [11]:
data.ecg.time

0            0.000
1            0.004
2            0.008
3            0.012
4            0.016
            ...   
266296    1065.184
266297    1065.188
266298    1065.192
266299    1065.196
266300    1065.200
Length: 266301, dtype: float64